# Data Wrangling - OpenStreetMap

## Area

I live and work in the San Jose now, so I just went to https://mapzen.com/data/metro-extracts/metro/san-jose_california/ and downloaded OSM XML file.

## Sample Dataset

In [146]:
import os
import collections
import pprint
import xml.etree.cElementTree as ET
import re
import codecs
import csv
import copy

In [147]:


OSM_FILE = "san-jose_california.osm"
SAMPLE_FILE = "sj_sample.osm"

k = 25

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

## Audit

Now I'm going to count tags and get an overview of the data sctructure.

In [3]:
OSM_FILE = "san-jose_california.osm"
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags
sf_tags = count_tags(OSM_FILE)
print sf_tags


{'node': 1346173, 'nd': 1572514, 'bounds': 1, 'member': 14717, 'tag': 704382, 'relation': 1463, 'way': 178588, 'osm': 1}


Now let's see the keys in the osm file and then identify which keys to be audited in this project.

In [4]:
def count_keys(tag, keys):
    k = tag.attrib['k']
    keys[k] += 1
    pass
def audit_keys(osm_file):
    keys = defaultdict(int)
    for i, elem in enumerate(get_element(osm_file, tags=['node', 'way'])):
        for tag in elem.iter('tag'):
            count_keys(tag, keys)
    
    return keys

In [5]:
from collections import defaultdict
import operator
keys = audit_keys(OSM_FILE)

sorted_keys = sorted(keys.items(), key=operator.itemgetter(1))
sorted_keys[::-1]

[('building', 101371),
 ('highway', 71624),
 ('name', 46907),
 ('tiger:county', 30664),
 ('tiger:name_base', 27703),
 ('tiger:name_type', 26638),
 ('tiger:cfcc', 24777),
 ('tiger:zip_left', 22975),
 ('tiger:zip_right', 22007),
 ('addr:housenumber', 21675),
 ('addr:street', 21266),
 ('oneway', 14879),
 ('tiger:reviewed', 14163),
 ('addr:postcode', 12669),
 ('service', 11057),
 ('height', 10955),
 ('source', 10726),
 ('tiger:tlid', 9730),
 ('tiger:source', 9663),
 ('tiger:separated', 9509),
 ('lanes', 8593),
 ('amenity', 7168),
 ('maxspeed', 6631),
 ('surface', 6298),
 ('cycleway', 5879),
 ('waterway', 5648),
 ('addr:city', 5595),
 ('created_by', 5440),
 ('layer', 5378),
 ('scvwd:ROUTEID', 5036),
 ('boat', 5031),
 ('turn:lanes', 4142),
 ('bicycle', 3973),
 ('landuse', 3597),
 ('tiger:name_direction_prefix', 3268),
 ('natural', 3213),
 ('leisure', 3044),
 ('source:maxspeed', 3015),
 ('tiger:upload_uuid', 2906),
 ('scvwd:FACILITY', 2576),
 ('access', 2466),
 ('ref', 2330),
 ('foot', 2288),

From the list above, we could see a large amounts of things to be audited. In this project, I will further explore 'addr:street' and 'addr:postcode'.

### addr:street

In [29]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

In [139]:

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type]+=1

In [140]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


In [141]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = collections.defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [142]:
street_types = audit(OSM_FILE)

In [145]:
pprint.pprint(dict(street_types))

{'0.1': 1,
 '1': 2,
 '109A': 1,
 '114': 1,
 '201': 1,
 '4A': 1,
 '6': 2,
 '7.1': 1,
 '81': 1,
 'Alameda': 22,
 'Ave': 39,
 'Avenue': 5520,
 'Barcelona': 17,
 'Bascom': 1,
 'Bellomy': 1,
 'Blvd': 20,
 'Boulevard': 571,
 'Boulvevard': 1,
 'CA': 2,
 'Cir': 1,
 'Circle': 235,
 'Court': 2280,
 'Ct': 1,
 'Dr': 9,
 'Drive': 5559,
 'East': 21,
 'Esquela': 1,
 'Expressway': 90,
 'Flores': 1,
 'Franklin': 1,
 'Hamilton': 1,
 'Highway': 4,
 'Hill': 2,
 'Hwy': 2,
 'Julian': 1,
 'Lane': 1062,
 'Ln': 2,
 'Loop': 42,
 'Luna': 2,
 'Madrid': 6,
 'Mall': 1,
 'Marino': 17,
 u'Monta\xf1a': 2,
 'Napoli': 18,
 'Oaks': 1,
 'Oro': 1,
 'Palamos': 19,
 'Parkway': 91,
 'Paviso': 1,
 'Place': 629,
 'Plaza': 88,
 'Portofino': 19,
 'Presada': 9,
 'Rd': 28,
 'Real': 207,
 'Road': 878,
 'Row': 4,
 'Saratoga': 1,
 'Seville': 8,
 'Sorrento': 16,
 'Sq': 1,
 'Square': 395,
 'St': 4,
 'Street': 781,
 'Terrace': 449,
 'Volante': 26,
 'Walk': 1,
 'Way': 1641,
 'West': 20,
 'Winchester': 1,
 'ave': 1,
 'court': 2,
 'robles':

We can see lots of street types and some of them are even not real street types. Also, we need mapping street types. For example, Rd = Road, St = Street and etc.

In [23]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Cove", "Alley", "Park", "Way", "Walk" "Circle", "Highway", 
            "Plaza", "Path", "Center", "Mission"]

mapping = { "Ave": "Avenue",
            "Ave.": "Avenue",
            "avenue": "Avenue",
            "ave": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Blvd,": "Boulevard",
            "Boulavard": "Boulevard",
            "Boulvard": "Boulevard",
            "Ct": "Court",
            "Dr": "Drive",
            "Dr.": "Drive",
            "E": "East",
            "Hwy": "Highway",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Pl": "Place",
            "Plz": "Plaza",
            "Rd": "Road",
            "Rd.": "Road",
            "St": "Street",
            "St.": "Street",
            "st": "Street",
            "street": "Street",
            "square": "Square",
            "Sq": "Square",
            "parkway": "Parkway"
            }

In [132]:
def update_name(name, mapping, expected):
    words = name.split()
    st_type = words[-1]
    if st_type in mapping:
        return name.replace(st_type, mapping[st_type])
    elif st_type in expected:
        return name
    else: return name

In [133]:
for i in street_types.keys():
    audit_list = mapping.keys()
    name = update_name(i, mapping, expected)
    print "Before updating:", i, "==>", "after updating:", name

Before updating: 6 ==> after updating: 6
Before updating: Boulevard ==> after updating: Boulevard
Before updating: Court ==> after updating: Court
Before updating: Winchester ==> after updating: Winchester
Before updating: Ln ==> after updating: Lane
Before updating: West ==> after updating: West
Before updating: Rd ==> after updating: Road
Before updating: 114 ==> after updating: 114
Before updating: 7.1 ==> after updating: 7.1
Before updating: Hill ==> after updating: Hill
Before updating: Way ==> after updating: Way
Before updating: ave ==> after updating: Avenue
Before updating: Circle ==> after updating: Circle
Before updating: East ==> after updating: East
Before updating: Alameda ==> after updating: Alameda
Before updating: Oaks ==> after updating: Oaks
Before updating: Highway ==> after updating: Highway
Before updating: Real ==> after updating: Real
Before updating: Expressway ==> after updating: Expressway
Before updating: robles ==> after updating: robles
Before updating: Pa

### addr:postcode

In [48]:
postcode = set()
for elem in get_element(OSM_FILE, tags = ("tag",)):
    if elem.get('k') == "addr:postcode":
        postcode.add(elem.get('v')) 
    elem.clear()

print postcode


set(['95014-1899', '95014-5398', '95014-3456', '95014-3457', '95014-1968', '95014-1960', '95014-1961', '95014-1962', '95014-1963', '95014-1964', '95014-1965', '95014-1966', '95037-4530', '95014-4664', '95014-4665', '95014-4666', '95014-4667', '95014-0549', '95014-0548', '95014-4662', '95014-4663', '95014-0545', '95014-0544', '95014-0547', '95014-0546', '95014-0541', '95014-0540', '95014-0543', '95014-0542', '95014-3012', '95014-3010', '95014-3011', '95014-3014', '95014-3015', '95014-3018', '95037-4222', '95037-4221', '95037-4220', '95037-4225', '95037-4224', '95014-2947', '95014-4449', '95014-2945', '95014-2944', '95014-2943', '95014-2942', '95014-2941', '95014-2940', '95014-4440', '95014-4441', '95014-4442', '95014-4443', '95014-4444', '95014-4445', '95014-3490', '95014-4447', 'CA 95113', '95014-3702', '95014-2911', 'CA 95116', '95014-3428', '95014-3029', '95014-0619', '95014-4431', '95014-3124', '95014-3125', '95014-3122', '95014-3123', '95014-3120', '95014-4430', '95014-0615', '9501

From the result, we could see that sometimes, postcode include "CA", let's remove it. Also, through Google search, I found that zipcode in San Jose will start from 94 or 95. 

In [52]:
def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if twoDigits != 94 or twoDigits != 95 or not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes


In [87]:
OSM_FILE.encode("utf-8")
zipcode = audit_zip(OSM_FILE)
pprint.pprint(dict(zipcode))

{'94': set(['94084',
            '94085',
            '94085-3010',
            '94086',
            '94086-6406',
            '94087',
            u'94087\u200e',
            '94088-3453',
            '94088-3707',
            '94089',
            '94089-2701',
            '94538',
            '94807']),
 '95': set(['95002',
            '95008',
            '95014',
            '95014-0200',
            '95014-0202',
            '95014-0236',
            '95014-0238',
            '95014-0240',
            '95014-030',
            '95014-0337',
            '95014-0353',
            '95014-0355',
            '95014-0358',
            '95014-0400',
            '95014-0431',
            '95014-0433',
            '95014-0434',
            '95014-0436',
            '95014-0437',
            '95014-0438',
            '95014-0439',
            '95014-0440',
            '95014-0444',
            '95014-0445',
            '95014-0446',
            '95014-0447',
            '95014-0448',
       

I'm going to remove CA, change all the zipcode to five digits and remove "CU" as well.

In [110]:

def update_zip(zipcode):
    zipChar = re.findall('[a-zA-Z]*', zipcode)
    if zipChar == "CU":
        return [None]
    if zipChar == "CA":
        updateZip = re.findall(r'\d+', zipcode)
        if (len(updateZip) > 0):
            return (re.findall(r'\d+', zipcode))[0]
    elif (len(re.findall(r'\d+', zipcode)) > 0):
        return (re.findall(r'\d+', zipcode))[0]



In [111]:

for street_type, ways in zipcode.iteritems():
    for name in ways:
        better_name = update_zip(name)
        print name, "=>", better_name

CA 95054 => 95054
CA 94035 => 94035
CA 95110 => 95110
CA 95113 => 95113
CA 95116 => 95116
CA 94085 => 94085
CA 94086 => 94086
95014-1899 => 95014
95014-3456 => 95014
95014-3457 => 95014
95014-1968 => 95014
95014-1960 => 95014
95014-1961 => 95014
95014-1962 => 95014
95014-1963 => 95014
95014-1964 => 95014
95014-1965 => 95014
95014-1966 => 95014
95037-4530 => 95037
95014-4664 => 95014
95014-4665 => 95014
95014-4666 => 95014
95014-4667 => 95014
95014-0549 => 95014
95014-0548 => 95014
95014-4662 => 95014
95014-4663 => 95014
95014-0545 => 95014
95014-0544 => 95014
95014-0547 => 95014
95014-0546 => 95014
95014-0541 => 95014
95014-0540 => 95014
95014-0543 => 95014
95014-0542 => 95014
95014-3012 => 95014
95014-3010 => 95014
95014-3011 => 95014
95014-3014 => 95014
95014-3018 => 95014
95037-4222 => 95037
95037-4221 => 95037
95037-4220 => 95037
95037-4225 => 95037
95037-4224 => 95037
95014-2947 => 95014
95014-4449 => 95014
95014-2945 => 95014
95014-2944 => 95014
95014-2943 => 95014
95014-2942 => 

## Parse Data to JSON

In [62]:
import json

In [63]:
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

In [67]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['id']= element.get('id')
        node['type'] = element.tag
        node['visible'] = element.get('visible')
        node['created'] ={}
        for key in CREATED:
            node['created'][key] = element.get(key)
        if element.get('lat') != None and element.get('lon') != None:
            node['pos'] = [ float(element.get('lat')), float(element.get('lon'))]
       
        address = {}
        node_refs = []
        for child in element:
            if child.tag =="tag":
                k = child.get('k')
                
                val = child.get('v')
                if k and val:
                    
                    if re.search('^addr:', k):
                       
                        key = re.findall('^addr:(.+$)', k)[0]
                        if key == "postcode":
                            val = update_zip(val)
                        elif key == "street":
                            val = update_name(val, mapping, expected)
                        else: pass
                        address[key]= val


                    else: 
                        node[k]= val
            
            elif child.tag == 'nd':
                node_refs.append(child.get('ref'))
        if address:
            node["address"] = address
        if node_refs:
            node["node_refs"] = node_refs
        return node
    else:
        return None

In [65]:
import time

def process_map(file_in, pretty = False):
    t0 = time.time()
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        
        for element in get_element(file_in, tags=("node", "way")):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
            
    t1 = time.time()

    total = t1-t0
    print "total process time:", total
    

In [112]:
process_map("san-jose_california.osm")

total process_map time: 51.6829998493


'san-jose_california.osm.json'

## Import Dataset to MongoDB

In Command: 1. mongod --repair 2. mongod 3. mongo 4. mongoimport --db OpenStreetMap --collection SanJose --file san-jose_california.osm.json

In [114]:
from pymongo import MongoClient
from pprint import pprint
import pymongo

client=MongoClient("localhost", 27017)
db = client.OpenStreetMap

In [115]:
collection = db.SanJose

## Explore Dataset

Now, let's explore the dataset. The size of the JSON file is about 350MB.

### Number of unique users

In [118]:
len(collection.distinct("created.user"))

1345

### Number of nodes and ways

In [119]:
collection.find({"type":"node"}).count()


1346165

In [120]:
collection.find({"type":"way"}).count()

178557

### Number of total documents

In [121]:
collection.count()

1524761

### Top 5 contribution users

In [122]:
top_user = collection.aggregate([
        {"$group": {"_id": "$created.user", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 5}
    ])

list(top_user)

[{u'_id': u'nmixter', u'count': 287912},
 {u'_id': u'mk408', u'count': 151363},
 {u'_id': u'Bike Mapper', u'count': 81192},
 {u'_id': u'samely', u'count': 77905},
 {u'_id': u'dannykath', u'count': 72273}]

### Top 5 postcode

In [124]:
postcode =  collection.aggregate( [
        {"$group": {"_id": "$address.postcode", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 5}
        ])
list(postcode)

[{u'_id': None, u'count': 1512093},
 {u'_id': u'95014', u'count': 9921},
 {u'_id': u'95037', u'count': 402},
 {u'_id': u'95070', u'count': 238},
 {u'_id': u'94087', u'count': 212}]

### Top 5 Amenity

In [125]:
amenity =  collection.aggregate( [
        {"$group": {"_id": "$amenity", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 5}
        ])
list(amenity)

[{u'_id': None, u'count': 1517593},
 {u'_id': u'parking', u'count': 1910},
 {u'_id': u'restaurant', u'count': 969},
 {u'_id': u'school', u'count': 534},
 {u'_id': u'fast_food', u'count': 494}]

### Amenity in each postcode

In [127]:
amenity_by_postcode =  collection.aggregate( [
        {"$match": {"amenity": {"$exists":1},
                    "address.postcode": {"$exists": 1}}},
        {"$group": {"_id": { "postcode": "$address.postcode",
                             "amenity": "$amenity"},
                    "amenity_count": {"$sum": 1}}},
        {"$match": {"amenity_count": {"$gt": 2}}},
        
        {"$sort": {"amenity_count": -1 }},
        {"$group": {"_id": "$_id.postcode",
                    "amenity_by_postcode": {"$push": {
                                                "amenity": "$_id.amenity",
                                                "count": "$amenity_count"
                    }},
                    "total_count":  {"$sum":1}
                   }},
        
        {"$sort": {"total_count":-1}},
        {"$limit": 5}
        ])

list(amenity_by_postcode)

[{u'_id': u'95014',
  u'amenity_by_postcode': [{u'amenity': u'restaurant', u'count': 34},
   {u'amenity': u'cafe', u'count': 8},
   {u'amenity': u'fast_food', u'count': 6},
   {u'amenity': u'bank', u'count': 5},
   {u'amenity': u'place_of_worship', u'count': 4},
   {u'amenity': u'fuel', u'count': 3},
   {u'amenity': u'school', u'count': 3}],
  u'total_count': 7},
 {u'_id': u'94087',
  u'amenity_by_postcode': [{u'amenity': u'restaurant', u'count': 10},
   {u'amenity': u'fast_food', u'count': 4},
   {u'amenity': u'dentist', u'count': 3},
   {u'amenity': u'bank', u'count': 3},
   {u'amenity': u'cafe', u'count': 3}],
  u'total_count': 5},
 {u'_id': u'95051',
  u'amenity_by_postcode': [{u'amenity': u'restaurant', u'count': 12},
   {u'amenity': u'bank', u'count': 4},
   {u'amenity': u'cafe', u'count': 4},
   {u'amenity': u'school', u'count': 3},
   {u'amenity': u'fast_food', u'count': 3}],
  u'total_count': 5},
 {u'_id': u'95129',
  u'amenity_by_postcode': [{u'amenity': u'restaurant', u'coun

Looks like 95014 area is the most common area with different kings of stores, banks, and schools. From Google search, 95014 is Cupertino area, where I usually go for lunch and dinner :)

## Other ideas about the datase

When I went through the database, there are a lot of information missing. For example, when I query the amenity, there are 1517593 none value.

In [128]:
amenity =  collection.aggregate( [
        {"$group": {"_id": "$amenity", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 5}
        ])
list(amenity)

[{u'_id': None, u'count': 1517593},
 {u'_id': u'parking', u'count': 1910},
 {u'_id': u'restaurant', u'count': 969},
 {u'_id': u'school', u'count': 534},
 {u'_id': u'fast_food', u'count': 494}]

In my opinion, it is probably because there are large amounts of data sources. Since each data source has different data structure, so sometimes, it may miss the "amenity" part. Also, we can see that most of the data sources are unknown!

In [129]:
source =  collection.aggregate( [
        {"$group": {"_id": "$source", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 20}
        ])
list(source)

[{u'_id': None, u'count': 1514035},
 {u'_id': u'bing', u'count': 4200},
 {u'_id': u'Yahoo', u'count': 1572},
 {u'_id': u'bing;survey', u'count': 1282},
 {u'_id': u'Bing', u'count': 911},
 {u'_id': u'photograph', u'count': 699},
 {u'_id': u'NHD', u'count': 516},
 {u'_id': u'http://www.consrv.ca.gov/dlrp/fmmp/products/Pages/DownloadGISdata.aspx',
  u'count': 357},
 {u'_id': u'yahoo', u'count': 215},
 {u'_id': u'Bing + ground verification', u'count': 116},
 {u'_id': u'USGS Geonames', u'count': 104},
 {u'_id': u'Tiger2009', u'count': 103},
 {u'_id': u'survey', u'count': 76},
 {u'_id': u'local knowledge', u'count': 45},
 {u'_id': u'http://openspace.org', u'count': 38},
 {u'_id': u'TIGER/Line\xae 2008 Place Shapefiles (http://www.census.gov/geo/www/tiger/)',
  u'count': 32},
 {u'_id': u'http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/twoeightynorth.pdf;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/twoeightysouth.pdf',
  u'count': 26},
 {u'_id': u'http://www.dot.ca.gov/hq/tr

## Conclusion

During this project, I audited, cleaned and explored San Jose OpenStreetMap data. I found two problems. One is the addr:street - street name is inconsistent such as "Rd" and "Road". I created a mapping for those stree names and delete unexpected street names. The other problems is the postcode. I identified unexpected "CA" and "CU", and updated all 9 digits to 5 digits, which I think is a good for further analysis.

When I looking at tag at the very beginning of the project, I found that there are hundreds of tags there. So one of my suggestions is to reduce the amount of tags and created more general tag names. In that case, we can save time to audit and clean the dataset. 

I also plan to audit and clean more tags in the near future to get more familiar with OpenStreetMap dataset. 